### MLP with Tensorboard 4

see:  https://github.com/nlintz/TensorFlow-Tutorials/blob/master/9_tensorboard.py

testing optimization loss function and gradient clipping

https://github.com/tensorflow/tensorflow/blob/c936c06a20b867eaa868e9f2c4fb53856391b18f/tensorflow/contrib/layers/python/layers/optimizers.py#L114

https://github.com/kapild/deeplearning/blob/8adb8452a8b16e15bdcec23153a78a3b1fa02f76/tensorflow/contrib/layers/python/layers/optimizers_test.py

will include test accuracy here

In [1]:
# A Multilayer Perceptron implementation example using TensorFlow library.
# This example is using the MNIST database of handwritten digits
# (http://yann.lecun.com/exdb/mnist/)=


# Author: Aymeric Damien
# Project: https://github.com/aymericdamien/TensorFlow-Examples/

In [2]:
import tensorflow as tf
import numpy as np
import input_data

In [3]:
%run infimnist.py

In [4]:
# Parameters
num_epochs = 5000
batch_size = 125

learning_rate = 0.001

In [5]:
# Network Parameters
n_hidden_1 = 1000 # 1st layer num features
n_hidden_2 = 500 # 2nd layer num features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

std_0 = 1.0/np.sqrt(n_input)
std_h1 = 1.0/np.sqrt(n_hidden_1)
std_h2 = 1.0/np.sqrt(n_hidden_2)


In [6]:
# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [7]:
def multilayer_perceptron(_X, _weights, _biases):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) 
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2'])) 
    return tf.matmul(layer_2, weights['out']) + biases['out']

In [8]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=std_0)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=std_h1)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=std_h2))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1], stddev=0.1)),
    'b2': tf.Variable(tf.random_normal([n_hidden_2], stddev=0.01)),
    'out': tf.Variable(tf.random_normal([n_classes], stddev=0.001))
}

In [9]:
# Construct model
mlp = multilayer_perceptron(x, weights, biases )

In [10]:
cost =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(mlp, y)) 
gs = tf.get_variable("global_step",[],trainable=False,initializer=tf.constant_initializer(0))
lr = tf.constant(learning_rate) 

In [11]:
train_op = tf.contrib.layers.optimize_loss(cost, global_step=gs, learning_rate=lr,optimizer="RMSProp")

In [12]:
#with tf.name_scope("training accuracy"):
pred = tf.equal(tf.argmax(mlp, 1), tf.argmax(y, 1)) # Count correct predictions
train_acc_op = tf.reduce_mean(tf.cast(pred, "float"))  # Cast boolean to float to average
test_acc_op = tf.reduce_mean(tf.cast(pred, "float"))  # Cast boolean to float to average
tf.scalar_summary("training accuracy", train_acc_op)
tf.scalar_summary("test accuracy", test_acc_op)

<tf.Tensor 'ScalarSummary_3:0' shape=() dtype=string>

In [13]:
infiminst = InfiMNIST()

In [14]:
!rm -rf ./logs/nn_logs_4

In [15]:
!ls logs

nn_logs_3


In [ ]:
with tf.Session() as sess:
    # create a log writer. run 'tensorboard --logdir=./logs/nn_logs'
    writer = tf.train.SummaryWriter("./logs/nn_logs_4", sess.graph) # for 0.8
    merged = tf.merge_all_summaries()

    tf.initialize_all_variables().run()

    for epoch in range(num_epochs):
        infiminst.next_epoch()
        mnist = input_data.read_data_sets(".", one_hot=True)
        
        trX, trY = mnist.train.images, mnist.train.labels
        for start, end in zip(range(0, len(trX), batch_size), range(125, len(trX), batch_size)):
            sess.run(train_op, feed_dict={x: trX[start:end], y: trY[start:end]})
                 
        summary, trn_acc = sess.run([merged, train_acc_op], feed_dict={x: trX, y: trY})
        writer.add_summary(summary, epoch)  
        
        teX, teY = mnist.test.images, mnist.test.labels
        tst_acc = test_acc_op.eval(feed_dict={x: teX, y: teY})
        
        print(epoch, trn_acc, tst_acc)

        writer.flush()

(0, 0.94936365, 0.9465)
(1, 0.95565456, 0.95859998)
(2, 0.97216362, 0.97100002)
(3, 0.97425455, 0.97369999)
(4, 0.97814548, 0.97329998)